In [1]:
import os
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# def import_data(path: str):
#     r"""This function:
#     - Imports the dataframe
#     - Removes unnecessary columns that will not be used in analysis
#     - Converts categorical data to dummy variables"""

# Read the CSV file
data = pd.read_csv("./data/unified_data_15_years.csv")

# Drop unnecessary columns
data = data.drop(columns=["system:index", ".geo", "biome"])

# Convert 'soil' and 'ecoreg' to categorical data
categorical = ["soil", "ecoreg"]
data[categorical] = data[categorical].astype("category")

# Create dummy variables for 'ecoreg' and 'soil'
data = pd.get_dummies(data, columns=["ecoreg", "soil"])



,agbd,age,all,b1,fallow,indig,last_LU,latitude,longitude,lulc_sum_15,...,soil_16,soil_18,soil_19,soil_22,soil_23,soil_24,soil_25,soil_28,soil_31,soil_32
0,92,11,0.459344,-218.25000,0,0,9,-21.553503,-43.831543,0,...,False,False,False,False,False,True,False,False,False,False
1,92,11,0.462767,-218.25000,0,0,9,-21.553503,-43.831272,0,...,False,False,False,False,False,True,False,False,False,False
2,229,19,0.368046,-207.16667,0,0,46,-21.645670,-43.870080,0,...,False,False,False,False,False,True,False,False,False,False
3,108,5,0.505563,-387.79166,0,1,15,4.278361,-61.133904,15,...,False,False,False,False,False,False,False,False,False,False
4,154,6,0.508987,-387.79166,0,1,15,4.278092,-61.133904,15,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19840,73,6,0.265906,-326.25000,0,0,21,-18.875536,-43.447780,8,...,False,False,False,False,False,False,False,False,False,True
19841,87,6,0.262197,-326.25000,0,0,21,-18.875805,-43.447780,8,...,False,False,False,False,False,False,False,False,False,True
19842,130,6,0.417689,-236.75000,0,0,21,-20.607307,-42.087643,0,...,False,False,False,False,False,False,False,True,False,False
19843,119,6,0.409130,-236.75000,0,0,21,-20.607576,-42.087914,0,...,False,False,False,False,False,False,False,True,False,False


In [10]:
# write the growth curve with yearly climatic data and permanent non-climatic data
def growth_curve(pars, data):
    r"""This function defines the growth function and parameter dictionary"""

    return (
        pars["B0"]
        + pars["A"] * (1 - np.exp(-pars["age"] * data["age"])) ** pars["theta"]
    )


# Initial guesses
initial_guess = [40, 80, 1.5, 1, 1, 1]  # B0, A, theta, sd, age


# Define a function to unpack parameters from a list
def unpack_parameters(params):
    return {
        "B0": params[0],
        "A": params[1],
        "theta": params[2],
        "sd": params[3],
        "age": params[4],
    }


# Modify the neg_log_likelihood function to unpack parameters
def neg_log_likelihood(params, data):
    params = unpack_parameters(params)
    likelihood = norm.logpdf(
        data["agbd"] - growth_curve(params, data), loc=0, scale=params["sd"]
    )
    return -np.sum(likelihood)


# Define bounds
bounds = [(0, 100), (50, 400), (0, None), (0, None), (0, None)]  # B0, A, theta, sd

# Call minimize with bounds
result = minimize(neg_log_likelihood, initial_guess, args=(data,), bounds=bounds)

ValueError: The number of bounds is not compatible with the length of `x0`.

In [9]:
# Extract the optimal parameters
optimal_params = result.x

# Generate predicted values
predicted = growth_curve(unpack_parameters(optimal_params), data)

# Compare predicted values to observed data
residuals = data["agbd"] - predicted

# Print the residuals
print(residuals)

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: nan
        x: [-1.256e+05 -1.248e+05  4.251e+04 -4.418e+03 -2.968e+05
             1.000e+00]
      nit: 2
      jac: [       nan        nan        nan        nan        nan
                   nan]
 hess_inv: [[ 1.000e+00 -2.320e-05 ... -5.010e-05  0.000e+00]
            [-2.320e-05  1.000e+00 ... -4.979e-05  0.000e+00]
            ...
            [-5.010e-05 -4.979e-05 ...  9.999e-01  0.000e+00]
            [ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]]
     nfev: 791
     njev: 113